# Explain

* Before performing parameter estimation for daily and monthly dataset, we interpolate daily and monthly dataset, and save interpolated datasets.
* To fill daily data from monthly data, we use `cubic` spline method, and extrapolate the outside of time boundary with (t2m[0], t2m[-1]]) values.

# Load modules

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from issmmodules import *
import numpy as np
import matplotlib.pyplot as plt
from cftime2datetime import *
import pandas
import scipy, scipy.io, scipy.interpolate

import copy
import tqdm

# Load Antarctica domain

In [ ]:
print('Load Antarctica mesh')
md = loadmodel_netcdf('../data/ANT_Mesh.nc')

# Do interpolation for daily dataset

In [ ]:
if 0:
    data_era5_day = {}
    for varname in ['d2m','t2m','msr','mtpr','msdwlwrf','msdwswrf','sp','wind2m']:
        print(f'Processing... {varname}')
        ds = interpCDO(md.mesh.lat, md.mesh.long,
                       f'../data/ERA5/day/era5_{varname}_1980,2000.nc', varname=varname)
        data_era5_day[varname] = ds[varname].values
    
    print('Save dataset in matlab format.')
    scipy.io.savemat('../data/Prepare/ANT_InterpERA5.mat',data_era5_day)

In [ ]:
if 0:
    data_era5_day = scipy.io.loadmat('../data/Prepare/ANT_InterpERA5.mat')

# Do interpolation for monthly dataset

In [ ]:
data_era5 = {}
for varname in ['d2m','t2m','msr','mtpr','msdwlwrf','msdwswrf','sp','wind2m']:
# for varname in ['t2m']:
    print(f'Processing... {varname}')
    print(f'-- interpolate montly dataset.')
    ds = interpCDO(md.mesh.lat, md.mesh.long,
                   f'../data/ERA5/day/era5_{varname}_1980,2000.nc', varname=varname, monavg=1)
    
    # okay, figure out times
    time = np.array([np.datetime64(t).astype('datetime64[ms]') for t in cftime2datetime(ds['time'])])

    tspc = pandas.date_range(start=datetime.datetime(1980,1,1),
                            end=datetime.datetime(2000,12,31))
    tspc = np.array([np.datetime64(t).astype('datetime64[ms]') for t in tspc])
    
    print(f'-- time to seconds with timedelta.')
    dtime = [t/np.timedelta64(1, 's') for t in (time - tspc[0])]
    dtspc = [t/np.timedelta64(1, 's') for t in (tspc - tspc[0])]

    print(f'-- fill monthly to daily dataset with cubic spline method.')
    tmp     = copy.deepcopy(ds[varname])
    tmp_day = np.zeros((len(tspc), md.mesh.numberofvertices))
    for n in tqdm.tqdm(range(md.mesh.numberofvertices),
                       bar_format='{l_bar}{bar:20}{r_bar}{bar:-10b}'):
        interp = scipy.interpolate.interp1d(dtime, tmp[:,n], bounds_error=False, kind='cubic',
                                   fill_value=(tmp[0,n], tmp[-1,n]))
        tmp_day[:,n] = interp(dtspc)
    data_era5[varname] = tmp_day[:]

print('Save dataset in matlab format.')
scipy.io.savemat('../data/Prepare/ANT_InterpERA5_Month.mat',data_era5)

In [ ]:
_ntime = 365*2
fig, ax = plt.subplots()
ax.plot(tspc[:_ntime], data_era5_day['t2m'][:_ntime,0], color='grey', alpha=.5)
ax.plot(tspc[:_ntime], data_era5['t2m'][:_ntime,0])